In [1]:
import pandas as pd
from pyvis.network import Network
from itertools import combinations
import networkx as nx
import re
from community import community_louvain
from copy import deepcopy

In [2]:
duchemin = pd.read_csv("data_files/DuChemin_Pieces_Phrases.csv")

In [3]:
duchemin.tail()

,PIECE ID,PHRASE ID,Phrase Start,Phrase Stop,Text,Rhyme,Column7,Column8,Column9,Column10,Column11,Column12
3161,DC1620,2.0,NaN,NaN,"Si vostre_amour ne me donne allegeance,",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3162,DC1620,3.0,NaN,NaN,"Mais de douleur vostre face jolye,",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3163,DC1620,4.0,NaN,NaN,Par quelque espoir me promet delivrance.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3164,DC1620,5.0,NaN,NaN,"Si j’ay refus j’en requerray vengeance,",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3165,DC1620,6.0,NaN,NaN,"Au dieu d’amour, qui a toute puissance.",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
duchemin.head()

,PIECE ID,PHRASE ID,Phrase Start,Phrase Stop,Text,Rhyme,Column7,Column8,Column9,Column10,Column11,Column12
0,DC0101,1.0,NaN,NaN,Qui souhaitez avoir tout le plaisir,a,NaN,NaN,NaN,NaN,NaN,NaN
1,DC0101,2.0,NaN,NaN,Qu'un amy peult vouloir honnestement,b,NaN,NaN,NaN,NaN,NaN,NaN
2,DC0101,3.0,NaN,NaN,Prenez exemple à mon chaste desir,a,NaN,NaN,NaN,NaN,NaN,NaN
3,DC0101,4.0,NaN,NaN,Et vous mirez en mon contentement.,b,NaN,NaN,NaN,NaN,NaN,NaN
4,DC0101,5.0,NaN,NaN,Mais qui vouldroit audacieusement,b,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
def remove_non_alpha_chars(text):
    return re.sub("\W*$", "", text)

In [6]:
duchemin.Text.apply(remove_non_alpha_chars).str.split()

0           [Qui, souhaitez, avoir, tout, le, plaisir]
1           [Qu'un, amy, peult, vouloir, honnestement]
2             [Prenez, exemple, à, mon, chaste, desir]
3             [Et, vous, mirez, en, mon, contentement]
4               [Mais, qui, vouldroit, audacieusement]
                             ...                      
3161     [Si, vostre_amour, ne, me, donne, allegeance]
3162          [Mais, de, douleur, vostre, face, jolye]
3163    [Par, quelque, espoir, me, promet, delivrance]
3164     [Si, j’ay, refus, j’en, requerray, vengeance]
3165    [Au, dieu, d’amour,, qui, a, toute, puissance]
Name: Text, Length: 3166, dtype: object

In [7]:
duchemin.Text.apply(remove_non_alpha_chars).str.split().str.get(-1)

0              plaisir
1         honnestement
2                desir
3         contentement
4       audacieusement
             ...      
3161        allegeance
3162             jolye
3163        delivrance
3164         vengeance
3165         puissance
Name: Text, Length: 3166, dtype: object

In [8]:
duchemin["rhyme_word"] = duchemin.Text.apply(remove_non_alpha_chars).str.split().str.get(-1)

In [9]:
duchemin

,PIECE ID,PHRASE ID,Phrase Start,Phrase Stop,Text,Rhyme,Column7,Column8,Column9,Column10,Column11,Column12,rhyme_word
0,DC0101,1.0,NaN,NaN,Qui souhaitez avoir tout le plaisir,a,NaN,NaN,NaN,NaN,NaN,NaN,plaisir
1,DC0101,2.0,NaN,NaN,Qu'un amy peult vouloir honnestement,b,NaN,NaN,NaN,NaN,NaN,NaN,honnestement
2,DC0101,3.0,NaN,NaN,Prenez exemple à mon chaste desir,a,NaN,NaN,NaN,NaN,NaN,NaN,desir
3,DC0101,4.0,NaN,NaN,Et vous mirez en mon contentement.,b,NaN,NaN,NaN,NaN,NaN,NaN,contentement
4,DC0101,5.0,NaN,NaN,Mais qui vouldroit audacieusement,b,NaN,NaN,NaN,NaN,NaN,NaN,audacieusement
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3161,DC1620,2.0,NaN,NaN,"Si vostre_amour ne me donne allegeance,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,allegeance
3162,DC1620,3.0,NaN,NaN,"Mais de douleur vostre face jolye,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,jolye
3163,DC1620,4.0,NaN,NaN,Par quelque espoir me promet delivrance.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,delivrance
3164,DC1620,5.0,NaN,NaN,"Si j’ay refus j’en requerray vengeance,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,vengeance


In [10]:
rhymes = duchemin.groupby(["PIECE ID", "Rhyme"]).rhyme_word.apply(list).reset_index()

In [11]:
rhymes

,PIECE ID,Rhyme,rhyme_word
0,DC0101,a,"[plaisir, desir]"
1,DC0101,b,"[honnestement, contentement, audacieusement, h..."
2,DC0101,c,"[tient, appartient]"
3,DC0102,a,"[plaist, desplaist]"
4,DC0102,b,"[marchander, demander]"
...,...,...,...
538,DC1313,c,"[diray, resjouiray]"
539,DC1414,a,"[secours, jours]"
540,DC1414,b,"[souspire, rire, m'escrire]"
541,DC1414,c,"[vouloir, douloir, non-chaloir]"


In [12]:
rhyme_pairs = rhymes.rhyme_word.apply(lambda x: list(combinations(x, 2)))

In [13]:
rhyme_pairs.iloc[1]

[('honnestement', 'contentement'),
 ('honnestement', 'audacieusement'),
 ('honnestement', 'humainement'),
 ('contentement', 'audacieusement'),
 ('contentement', 'humainement'),
 ('audacieusement', 'humainement')]

In [14]:
unique_rhyme_pairs = rhyme_pairs.explode().dropna().unique()
unique_rhyme_pairs

array([('plaisir', 'desir'), ('honnestement', 'contentement'),
       ('honnestement', 'audacieusement'), ...,
       ('vouloir', 'non-chaloir'), ('douloir', 'non-chaloir'),
       ('supporte', 'porte')], dtype=object)

In [15]:
pd.Series(unique_rhyme_pairs).isna().sum()

0

In [16]:
def add_communities(G):
    G = deepcopy(G)
    partition = community_louvain.best_partition(G)
    nx.set_node_attributes(G, partition, "group")
    return G

In [17]:
G = nx.Graph()
G.add_edges_from(unique_rhyme_pairs)
G = add_communities(G)

In [18]:
pyvis_graph = Network(notebook=False, width="1600px", height="900px", bgcolor="black", font_color="white")

In [19]:
pyvis_graph.from_nx(G)

In [20]:
pyvis_graph.show("rhymes.html")

In [21]:
unique_rhyme_pairs

array([('plaisir', 'desir'), ('honnestement', 'contentement'),
       ('honnestement', 'audacieusement'), ...,
       ('vouloir', 'non-chaloir'), ('douloir', 'non-chaloir'),
       ('supporte', 'porte')], dtype=object)